In [1]:
import os, sys
import tensorflow as tf
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import albumentations as A
from PIL import Image
import cv2
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
FEATURE_DESCRIPTION = {
    'filename': tf.io.VarLenFeature(tf.string),
    'shape' : tf.io.FixedLenFeature([3], tf.int64),
    'image' : tf.io.FixedLenFeature([], tf.string),
    'bbxs' : tf.io.VarLenFeature(tf.int64), 
    'faces' : tf.io.FixedLenFeature([1], tf.int64),
}

bbx_params = A.BboxParams('albumentations', min_area=1, min_visibility=-2, label_fields=None, check_each_transform=False)
transform = A.Compose([
    A.Resize(IMAGE_SIZES[0], IMAGE_SIZES[1]),
], bbox_params = bbx_params)

def prepare_bbxs(bbxs, width, height):
    bbx_to_transform = []
    for bbx in bbxs:
        temp = bbx
        temp[2] += temp[0]
        temp[2] /= width
        
        temp[3] += temp[1]
        temp[3] /= height

        temp[0] /= width
        temp[1] /= height

        if temp[0] > 1 or temp[1] > 1 or temp[2] > 1 or temp[2] <= temp[0] or temp[3] > 1 or temp[3] <= temp[1]:
            continue
        
        temp2 = temp.copy()
        # temp2[0] = temp[1]
        # temp2[1] = temp[0]
        # temp2[2] = temp[3]
        # temp2[3] = temp[2]

        temp2.append('x')

        bbx_to_transform.append(temp2)
    return bbx_to_transform


def decode_img(img, bbxs = [], resize=False):
    image = tf.image.decode_jpeg(img , channels=3)

    height, width, channels = image.shape
    image = tf.cast(image, tf.float32)

    bbxs_return = []
    if resize:
        image = tf.image.resize(image, [IMAGE_SIZES[0], IMAGE_SIZES[1]])
        image = image.numpy()
        bbx_to_transform = prepare_bbxs(bbxs, width, height)
        if(len(bbxs) > 0):
            transformed = transform(image=image, bboxes=bbx_to_transform)
            bbxs_temp = transformed['bboxes']

            for bbx in bbxs_temp:
                bbxs_return.append([
                     round(bbx[0] * IMAGE_SIZES[0]),
                     round(bbx[1] * IMAGE_SIZES[1]),
                     round((bbx[2] - bbx[0]) * IMAGE_SIZES[0]),
                     round((bbx[3] - bbx[1]) * IMAGE_SIZES[1])
                ])
        
        else: 
            transformed = transform(image=image)
            bbxs_return.append([])

        image = tf.convert_to_tensor(transformed['image'])
    else:
        return image
    return image, bbxs_return


def parse_function(example_proto):
  # Parse the input `tf.Example` proto using the dictionary above.
  example = tf.io.parse_single_example(example_proto, FEATURE_DESCRIPTION)
  image = decode_img(example['image'])
  image = tf.cast(image, tf.uint8)
  #   print(example['bbxs'])
  bbxs = tf.cast(example['bbxs'], tf.int64)
  print(tf.cast(example['filename'], tf.string))
  if isinstance(bbxs, tf.SparseTensor):
      bbxs = tf.sparse.to_dense(bbxs)
      
  return image, bbxs

In [4]:
path = os.path.join(INSTALLATION_PATH, "test_scaled_down.jpeg")
src = os.path.join(TRAIN_IMAGES, "0--Parade\\0_Parade_Parade_0_904.jpg")

with tf.io.gfile.GFile(src, 'rb') as fid:
    image_bytes = fid.read()
    image_bytes, bbxs = decode_img(image_bytes, bbxs=[[361, 98, 263, 339]], resize=True)
    print(bbxs)
    image_bytes = tf.cast(image_bytes, tf.uint8)
    tf.io.write_file(path, tf.io.encode_jpeg(image_bytes))

    with tf.io.gfile.GFile(path, 'rb') as fid2:
        img_tensor = decode_img(fid2.read())
        img_tensor = tf.cast(img_tensor, tf.uint8)
        # img_tensor = tf.io.encode_jpeg(img_tensor)

        img = Image.fromarray(img_tensor.numpy())
        
        img.show()
        # plt.imshow(img_tensor.numpy())

    # img = mpimg.imread(path)
    # plt.imshow(img)

[[79, 15, 58, 53]]


In [8]:

def draw_bounding_boxes_from_tf_record(temppath, window_name):
    c = 0
    for record in tf.compat.v1.python_io.tf_record_iterator(OUTPUT_TRAIN_TFRECORD):
        c+=1
        if c < 2: continue
        image, _bbxs = parse_function(record)
        image = tf.cast(image, tf.uint8)
        
        _bbxs = _bbxs.numpy()
        bbxs =  [_bbxs[x:x+4] for x in range(0, len(_bbxs), 4)]

        tf.io.write_file(tempath, tf.io.encode_jpeg(image))

        img = cv2.imread(tempath)

        for bbx in bbxs:
            x1, y1 = bbx[0], bbx[1]

            if x1 == 0 and y1 == 0:
                continue

            x2, y2 = bbx[0] + bbx[2], bbx[1] + bbx[3]
            print(bbx)

            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 1)

        cv2.imshow(window_name, img)

        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # img.show()
        break

In [9]:
tempath = os.path.join(INSTALLATION_PATH, "scaled_down_2223.jpg")
window_name = "xxx"
draw_bounding_boxes_from_tf_record(tempath, window_name)

SparseTensor(indices=tf.Tensor([[0]], shape=(1, 1), dtype=int64), values=tf.Tensor([b'0--Parade/0_Parade_Parade_0_904.jpg'], shape=(1,), dtype=string), dense_shape=tf.Tensor([1], shape=(1,), dtype=int64))
[79 15 58 53]
